### Comment and Unit Test Generater 

The requirement: 
* use an LLM to generate docstring and comments for Python code
* use an LLM to generate unit test

This is my week 4 day 5 project.

In [ ]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [ ]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [ ]:
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
CODE_QWEN_URL = "https://g39mbjooiiwkbgyz.us-east-1.aws.endpoints.huggingface.cloud"

In [ ]:
def system_prompt_for_comment():
    system="""
    You are a Python documentation expert. When writing documentation:
    - Follow PEP 257 and Google docstring style guidelines
    - Write clear, concise explanations
    - Include practical examples
    - Highlight edge cases and limitations
    - Use type hints in docstrings
    - Add inline comments only for complex logic
    - Never skip documenting parameters or return values
    - Validate that all documentation is accurate and complete
    """
    return system

In [ ]:
def system_prompt_for_unit_test():
    system="""
    You are an expert Python testing engineer who specializes in creating comprehensive unit tests. Follow these principles:
    - Use pytest as the testing framework
    - Follow the Arrange-Act-Assert pattern
    - Test both valid and invalid inputs
    - Include edge cases and boundary conditions
    - Write descriptive test names that explain the scenario being tested
    - Create independent tests that don't rely on each other
    - Use appropriate fixtures and parametrize when needed
    - Add clear comments explaining complex test logic
    - Cover error cases and exceptions
    - Achieve high code coverage while maintaining meaningful tests
    """
    return system

In [ ]:
def user_prompt_for_comment(code):
    user = f"""
        Please document this Python code with:
        
        1. A docstring containing:
        - A clear description of purpose and functionality
        - All parameters with types and descriptions
        - Return values with types
        - Exceptions that may be raised
        - Any important notes or limitations
        
        2. Strategic inline comments for:
        - Complex algorithms or business logic
        - Non-obvious implementation choices
        - Performance considerations
        - Edge cases
        
        Here's the code to document:
        \n{code}
        """
    return user;

In [ ]:
def user_prompt_for_unit_test(code):
    user = f"""
        Please generate unit tests for the following Python code. Include:
        
        1. Test cases for:
        - Normal/expected inputs
        - Edge cases and boundary values
        - Invalid inputs and error conditions
        - Different combinations of parameters
        - All public methods and functions
        
        2. For each test:
        - Clear test function names describing the scenario
        - Setup code (fixtures if needed)
        - Test data preparation
        - Expected outcomes
        - Assertions checking results
        - Comments explaining complex test logic
        
        3. Include any necessary:
        - Imports
        - Fixtures
        - Mock objects
        - Helper functions
        - Test data generators
        
        Here's the code to test:
        \n{code}
    """
    return user

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

In [ ]:
def messages_for_comment(python):
    return [
        {"role": "system", "content": system_prompt_for_comment()},
        {"role": "user", "content": user_prompt_for_comment(python)}
    ]

In [ ]:
def messages_for_unit_test(python):
    return [
        {"role": "system", "content": system_prompt_for_unit_test()},
        {"role": "user", "content": user_prompt_for_unit_test(python)}
    ]

In [ ]:
def stream_comment_gpt(code):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for_comment(code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        #print(fragment, end='', flush=True)
        yield reply.replace('```','') 
    

In [ ]:
stream_comment_gpt(pi)

In [ ]:
def stream_unit_test_gpt(code):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for_unit_test(code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        #print(fragment, end='', flush=True)
        yield reply.replace('```','')

In [ ]:
stream_unit_test_gpt(pi)

In [ ]:
def stream_comment_claude(code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_prompt_for_comment(),
        messages=[{"role": "user", "content": user_prompt_for_comment(code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            #print(text, end="", flush=True)
            yield reply.replace('```','')

In [ ]:
def stream_unit_test_claude(code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_prompt_for_unit_test(),
        messages=[{"role": "user", "content": user_prompt_for_unit_test(code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            #print(text, end="", flush=True)
            yield reply.replace('```','')

In [ ]:
stream_comment_claude(pi)

In [ ]:
stream_unit_test_claude(pi)

In [ ]:
def stream_comment_model(model, model_url, code):
    tokenizer = AutoTokenizer.from_pretrained(model)
    messages = messages_for_comment(code)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(model_url, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=5000)
    result = ""
    for r in stream:
        #print(r.token.text, end = "")
        result += r.token.text
        yield result    
    

In [ ]:
def stream_unit_test_model(model, model_url, code):
    tokenizer = AutoTokenizer.from_pretrained(model)
    messages = messages_for_unit_test(code)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(model_url, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)
    result = ""
    for r in stream:
        #print(r.token.text, end = "")
        result += r.token.text
        yield result    
    

In [ ]:
stream_comment_model(code_qwen, CODE_QWEN_URL, pi)

In [ ]:
stream_unit_test_model(code_qwen, CODE_QWEN_URL, pi)

In [ ]:
def comment_code(python, model):
    if model=="GPT":
        result = stream_comment_gpt(python)
    elif model=="Claude":
        result = stream_comment_claude(python)
    elif model=="CodeQwen":
        result = stream_comment_model(code_qwen, CODE_QWEN_URL, python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far     

In [ ]:
def get_unit_test(python, model):
    if model=="GPT":
        result = stream_unit_test_gpt(python)
    elif model=="Claude":
        result = stream_unit_test_claude(python)
    elif model=="CodeQwen":
        result = stream_unit_test_model(code_qwen, CODE_QWEN_URL, python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far     

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        result = gr.Textbox(label="Result code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude","CodeQwen"], label="Select model", value="GPT")
    with gr.Row():
        comment_button = gr.Button("Comment code")
    with gr.Row():
        unit_test_button = gr.Button("Unit Test code")
        
    comment_button.click(comment_code, inputs=[python, model], outputs=[result])
    unit_test_button.click(get_unit_test, inputs=[python, model], outputs=[result])
ui.launch(inbrowser=True)